# Sydney COVID-19 cases by postcode

<a href="https://colab.research.google.com/github/maxim75/data-visualization/blob/master/notebooks/COVID_data_2.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" /><br/>Open in Colab</a>


In [1]:
!pip install geopandas contextily mapclassify 

In [2]:
import geopandas as gpd

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
import pandas as pd
import geopandas as gpd
import mapclassify as mc
import datetime

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.dates as mdates
import matplotlib.patches as mpatches

import contextily as ctx
import pyproj 

In [4]:
start_date = "2021-06-15"
end_date = datetime.datetime.now().strftime("%Y-%m-%d")

# Data source 
# https://data.nsw.gov.au/nsw-covid-19-data/cases
confirmed_cases_url = "https://data.nsw.gov.au/data/dataset/aefcde60-3b0c-4bc0-9af1-6fe652944ec2/resource/21304414-1ff1-4243-a5d2-f52778048b29/download/confirmed_cases_table1_location.csv"

# load data from CSV file
cases_df = pd.read_csv(confirmed_cases_url)
cases_df.notification_date = pd.to_datetime(cases_df.notification_date)

# remove last date from data since it is incomplete
cases_df = cases_df[cases_df.notification_date < cases_df.notification_date.max()]

# filter by start_date and end_date
cases_df = cases_df[(cases_df.notification_date >= start_date) & (cases_df.notification_date <= end_date)]
cases_df

,notification_date,postcode,lhd_2010_code,lhd_2010_name,lga_code19,lga_name19
5431,2021-06-15,2207,X720,South Eastern Sydney,10500.0,Bayside (A)
5432,2021-06-16,2026,X720,South Eastern Sydney,18050.0,Waverley (A)
5433,2021-06-16,Masked,X720,South Eastern Sydney,18050.0,Waverley (A)
5434,2021-06-16,NaN,NaN,NaN,NaN,NaN
5435,2021-06-16,2050,X700,Sydney,17200.0,Sydney (C)
...,...,...,...,...,...,...
18722,2021-08-22,2760,X750,Nepean Blue Mountains,16350.0,Penrith (C)
18723,2021-08-22,2161,X740,Western Sydney,12380.0,Cumberland (A)
18724,2021-08-22,2192,X700,Sydney,11570.0,Canterbury-Bankstown (A)
18725,2021-08-22,2565,X710,South Western Sydney,11500.0,Campbelltown (C) (NSW)


In [5]:
poa_population = pd.read_csv("https://raw.githubusercontent.com/maxim75/data-visualization/master/notebooks/data/census/poa_population.csv").set_index("Unnamed: 0")
# Set population for postcode manually (suburb population has grown significantly since 2016 Census)
poa_population.loc['POA2174']['Tot_P_P'] = 5409
poa_population

,ASGS_Code_2016,Tot_P_P
Unnamed: 0,,
POA2000,2000,27411
POA2006,2006,1259
POA2007,2007,8845
POA2008,2008,11712
POA2009,2009,12813
...,...,...
POA2877,2877,3883
POA2878,2878,372
POA2879,2879,638


In [6]:
poa_total_cases_df = cases_df.groupby(by=["postcode"]).size().to_frame("case_count").reset_index()
poa_total_cases_df = poa_total_cases_df[poa_total_cases_df.postcode != "Masked"]
poa_total_cases_df = poa_total_cases_df.astype({'postcode': 'int32'})
poa_total_cases_df

,postcode,case_count
0,1781,1
1,1871,1
2,2000,27
3,2007,7
4,2008,2
...,...,...
294,2850,7
295,2866,1
296,2870,6
297,2880,4


In [7]:
poa_total_cases_df = pd.merge(poa_total_cases_df, poa_population[["ASGS_Code_2016", "Tot_P_P"]], 
                              left_on="postcode", right_on="ASGS_Code_2016")[["postcode", "case_count", "Tot_P_P"]]

poa_total_cases_df = poa_total_cases_df.rename(columns={ "Tot_P_P": "population"})
poa_total_cases_df

poa_total_cases_df["cases_per_1000"] = poa_total_cases_df.case_count / poa_total_cases_df.population * 1000

poa_total_cases_df = poa_total_cases_df.sort_values("cases_per_1000", ascending=False)

poa_total_cases_df.head(20)

,postcode,case_count,population,cases_per_1000
285,2836,33,891,37.037037
95,2139,1,57,17.543860
116,2161,493,31418,15.691642
133,2179,102,6530,15.620214
120,2165,601,42444,14.159834
115,2160,447,35973,12.425986
286,2838,3,247,12.145749
134,2190,304,25566,11.890792
139,2195,314,27154,11.563674
276,2818,10,871,11.481056


In [8]:
postcodes_df = gpd.read_file("https://raw.githubusercontent.com/maxim75/data-visualization/master/notebooks/data/geo/greater_sydney_postcodes.json").to_crs(epsg=4326)
postcodes_df = postcodes_df.astype({'POA_NAME16': 'int32'})
postcodes_df

,POA_CODE16,POA_NAME16,AREASQKM16,geometry
0,2006,2006,0.6780,"POLYGON ((151.19142 -33.88470, 151.19162 -33.8..."
1,2007,2007,0.5588,"POLYGON ((151.19469 -33.88090, 151.19461 -33.8..."
2,2008,2008,0.8450,"POLYGON ((151.19399 -33.88655, 151.19441 -33.8..."
3,2009,2009,0.9325,"POLYGON ((151.18869 -33.86635, 151.18908 -33.8..."
4,2010,2010,2.1733,"POLYGON ((151.21075 -33.87860, 151.21102 -33.8..."
...,...,...,...,...
202,2766,2766,27.2385,"POLYGON ((150.83116 -33.76492, 150.83065 -33.7..."
203,2767,2767,10.1088,"POLYGON ((150.85568 -33.76181, 150.85563 -33.7..."
204,2768,2768,9.0193,"POLYGON ((150.91324 -33.72664, 150.91280 -33.7..."
205,2769,2769,4.6535,"POLYGON ((150.89422 -33.70375, 150.89448 -33.7..."


In [9]:
poa_total_cases_df = pd.merge(postcodes_df, poa_total_cases_df, left_on="POA_NAME16", right_on="postcode")

In [10]:
epsg3857 = pyproj.Proj('epsg:3857')
epsg4326 = pyproj.Proj('epsg:4326')

proj_transformer_4326_to_3857 = pyproj.Transformer.from_crs("epsg:4326", "epsg:3857")

sydney_bb = {
    "north": -33.62,
    "south": -34.086,
    "west": 150.6,
    "east": 151.38,
}

sydney_bb_epsg3857 = {}
sydney_bb_epsg3857["west"], sydney_bb_epsg3857["north"]  = proj_transformer_4326_to_3857.transform(sydney_bb["north"],sydney_bb["west"])
sydney_bb_epsg3857["east"], sydney_bb_epsg3857["south"],  = proj_transformer_4326_to_3857.transform(sydney_bb["south"],sydney_bb["east"])

In [51]:
#cases_df[cases_df.postcode == "2165"]

daily_cases_df = cases_df[cases_df.postcode != "Masked"].groupby(by=["postcode", "notification_date"]).size().to_frame("case_count").reset_index()

daily_cases_df["postcode"] = daily_cases_df.postcode.astype("int64")
daily_cases_df

total_daily_cases_df = cases_df[cases_df.postcode != "Masked"].groupby(by=["notification_date"]).size().to_frame("case_count").reset_index()
total_daily_cases_df

,notification_date,case_count
0,2021-06-15,1
1,2021-06-16,3
2,2021-06-17,1
3,2021-06-18,7
4,2021-06-19,7
...,...,...
64,2021-08-18,644
65,2021-08-19,774
66,2021-08-20,789
67,2021-08-21,838


In [12]:
dates = pd.date_range(start_date, end_date)
postcodes = daily_cases_df.groupby(by="postcode").size().index.astype("int64")
dates.to_series()
#pd.merge(dates, postcodes)
postcode_date = dates.to_frame().merge(postcodes.to_frame(), how='cross').rename(columns={0:"notification_date"}).set_index(["notification_date", "postcode"])

cases_date_postcode = postcode_date.merge(daily_cases_df.set_index(["notification_date", "postcode"]), left_index=True, right_index=True, how="left").reset_index()
cases_date_postcode = cases_date_postcode.fillna(0)

cases_date_postcode = pd.merge(cases_date_postcode, poa_population[["ASGS_Code_2016", "Tot_P_P"]], 
                              left_on="postcode", right_on="ASGS_Code_2016")[["notification_date", "postcode", "case_count", "Tot_P_P"]]

cases_date_postcode["cases_per_1000"] = cases_date_postcode.case_count / cases_date_postcode.Tot_P_P * 1000
cases_date_postcode["notification_date"] = cases_date_postcode["notification_date"].astype("string")
cases_date_postcode

,notification_date,postcode,case_count,Tot_P_P,cases_per_1000
0,2021-06-15,2000,0.0,27411,0.000000
1,2021-06-16,2000,0.0,27411,0.000000
2,2021-06-17,2000,0.0,27411,0.000000
3,2021-06-18,2000,0.0,27411,0.000000
4,2021-06-19,2000,0.0,27411,0.000000
...,...,...,...,...,...
20869,2021-08-20,2880,1.0,18276,0.054717
20870,2021-08-21,2880,1.0,18276,0.054717
20871,2021-08-22,2880,1.0,18276,0.054717
20872,2021-08-23,2880,0.0,18276,0.000000


In [13]:
cases_date_postcode_pivot = cases_date_postcode.pivot(index='notification_date', columns="postcode", values="cases_per_1000")
cases_date_postcode_pivot_mean = cases_date_postcode_pivot.rolling(window=7, min_periods=7).mean().iloc[6:]
cases_date_postcode_pivot_mean = cases_date_postcode_pivot_mean.transpose()
cases_date_postcode_pivot_mean = pd.merge(postcodes_df, cases_date_postcode_pivot_mean, left_on="POA_NAME16", right_on="postcode")
cases_date_postcode_pivot_mean = gpd.GeoDataFrame(cases_date_postcode_pivot_mean, geometry="geometry")

In [14]:
values_df = cases_date_postcode_pivot_mean[cases_date_postcode_pivot_mean.columns[4:]]
q10 = mc.Pooled(values_df, classifier="FisherJenks", k=10)

In [77]:
size_x = 3840
size_y = 2160
dpi = 200
render_video_frames = True

def get_legend_elements(classifier):
    cmap = matplotlib.cm.get_cmap('YlOrRd')
    legend_elements = []
    idx = 0
    classes_count = len(classifier.get_legend_classes())
    for lclass in classifier.get_legend_classes():
        rgba = cmap(idx/classes_count)
        idx += 1
        legend_elements.append(mpatches.Patch(facecolor=rgba, edgecolor='#cccccc', label=lclass))
    return legend_elements


def show_map(show_date, idx, render_to_file):
    fig = plt.figure(constrained_layout=True, figsize=(size_x/dpi, size_y/dpi), dpi=dpi)
    #gs = GridSpec(2, 2, figure=fig)
    gs = matplotlib.gridspec.GridSpec(3, 5, wspace=0.5, hspace=0.02)
    
    ax_map = fig.add_subplot(gs[:, 0:4])
    ax_map.set_axis_off()

    ax_map.set_xlim(sydney_bb_epsg3857["west"], sydney_bb_epsg3857["east"])
    ax_map.set_ylim(sydney_bb_epsg3857["south"], sydney_bb_epsg3857["north"])

    cases_date_postcode_pivot_mean.to_crs(epsg=3857).assign(cl=q10.col_classifiers[0](cases_date_postcode_pivot_mean[show_date])).plot(
        ax=ax_map, cmap='YlOrRd', column='cl', 
        alpha=0.7, legend=False, categorical=True, vmax=10, vmin=0,
        legend_kwds={'loc': 'lower right'}   #, 'labels': q10.get_legend_classes()
    ) 

    ctx.add_basemap(ax_map, zoom=11, source=ctx.providers.Stamen.TonerLines, alpha=.3)
    ctx.add_basemap(ax_map, zoom=11, source=ctx.providers.Stamen.TonerLabels)

    ax_map.legend(handles=get_legend_elements(q10.global_classifier), loc='lower right', labelspacing=.2, handlelength=1, title="Cases per day per 1000 people")
    ax_map.text(x=sydney_bb_epsg3857["west"]+0.01e5, y=sydney_bb_epsg3857["north"]-0.4e4, s=datetime.datetime.strptime(show_date, "%Y-%m-%d").strftime("%-d %b %Y"), fontsize=25)
    ax_map.text(x=sydney_bb_epsg3857["west"]+0.01e5, y=sydney_bb_epsg3857["south"]-0.4e4, s="Created by Maksym Kozlenko using NSW COVID-19 cases data https://data.nsw.gov.au/nsw-covid-19-data/cases", fontsize=14)

    ax_case = fig.add_subplot(gs[2, 0:1], alpha=.4)
    ax_case.set_title("Total cases")
    ax_case.plot("notification_date", "case_count", data=total_daily_cases_df[total_daily_cases_df.notification_date <= show_date])
    ax_case.get_xaxis().set_visible(False)
    
    if render_to_file:
        plt.savefig(f"images/myfig_{idx:04d}.png", dpi=dpi)
        plt.close()

if render_video_frames:
    idx = 0
    for date in dates[6:]:
        show_map(str(date)[:10], idx, True)
        idx += 1
else:
    show_map(str(dates[-10])[:10], 0, False)

/opt/conda/lib/python3.9/site-packages/geopandas/plotting.py:909: UserWarning: There are no gridspecs with layoutgrids. Possibly did not call parent GridSpec with the "figure" keyword
  plt.draw()
/opt/conda/lib/python3.9/site-packages/contextily/plotting.py:294: UserWarning: There are no gridspecs with layoutgrids. Possibly did not call parent GridSpec with the "figure" keyword
  draw()
/tmp/ipykernel_54/3512053638.py:48: UserWarning: There are no gridspecs with layoutgrids. Possibly did not call parent GridSpec with the "figure" keyword
  plt.savefig(f"images/myfig_{idx:04d}.png", dpi=dpi)
/opt/conda/lib/python3.9/site-packages/geopandas/plotting.py:909: UserWarning: There are no gridspecs with layoutgrids. Possibly did not call parent GridSpec with the "figure" keyword
  plt.draw()
/opt/conda/lib/python3.9/site-packages/contextily/plotting.py:294: UserWarning: There are no gridspecs with layoutgrids. Possibly did not call parent GridSpec with the "figure" keyword
  draw()
/tmp/ipyke

In [ ]:
cases_date_postcode_pivot_mean[["POA_NAME16", "2021-08-18"]].sort_values(by="2021-08-18").tail(20)

Generate video from images:

    ffmpeg -framerate 2 -i images/myfig_%4d.png -c:v h264 -r 20 -s 3840x2160 ./covid_sydney.mp4